In [ ]:
import os
print(os.getcwd())

d:\pythonProject\Do-Not-Touch-It


In [33]:
# pip install -U sentence-transformers

In [34]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", show_progress=True)
print(embeddings.cache_folder)

None


In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.vectorstores import InMemoryVectorStore
# from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# pip install -U  docx2txt

from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./example.docx") 

data = loader.load()

data

[Document(metadata={'source': './example.docx'}, page_content='Hello everyone,\n\nMy name is Serein, and I am excited to be here today. I am currently an Engineer at Shenzhen where I [briefly describe your role or responsibilities].\n\nI graduated from [Your University] with a degree in [Your Major]. During my studies, I developed a strong passion for [specific interests related to your field or profession], which has influenced my career path.\n\nIn my spare time, I enjoy [mention hobbies or interests], and I also love to [any other personal interests or activities]. I believe that continuous learning is important, and I am always looking for opportunities to grow and expand my skills.\n\nThank you for this opportunity to introduce myself. I look forward to connecting with all of you!')]

In [37]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(data)

In [38]:
vectorstore = InMemoryVectorStore(embedding=embeddings)
vectorstore.add_documents(splits)
retriever = vectorstore.as_retriever()

Batches: 100%|██████████| 1/1 [00:00<00:00, 10.15it/s]


In [39]:
# import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [40]:

### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


In [41]:

### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [42]:

### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [43]:
conversational_rag_chain.invoke(
    {"input": "What's the content of this passage?"},
    config={
        "configurable": {"session_id": "a_new_chat"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

Batches: 100%|██████████| 1/1 [00:00<00:00, 62.51it/s]


'The passage is an introduction from Serein, an engineer based in Shenzhen. It includes their current role, educational background with specific major and university, specific interests developed during their studies, hobbies and personal interests, and a statement about the value of continuous learning and professional growth.'